In [1]:
import csv
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapefile
import pickle
import os

In [7]:
# open risk file
with open ("Dane_adjusted_risk.pk","rb") as f:
    risk = pickle.load(f)
# open speed file
with open ("Dane_speed_dict.pk","rb") as f:
    speed = pickle.load(f)

In [24]:
# get functional class file
fnct_cls_path = os.path.join("..","..","transportation_data","dane-county","dane_wislr_rdwy_link_fnct_cls.csv")
with open(fnct_cls_path,'r') as f:
    fnct_cls = list(csv.reader(f))
fnct_header = fnct_cls[0]
fnct_cls = fnct_cls[1:]  

# roadID to reference tuple {roadID : (ref1,ref2)}

roadToRef = {}
for r in fnct_cls:
    roadToRef[int(r[fnct_header.index('RDWY_LINK_ID')])] = \
             (r[fnct_header.index('REF_SITE_FROM_ID')], r[fnct_header.index('REF_SITE_TO_ID')])

In [30]:
# read rte file
# rte file stores the road information of a roadlink. Note that each road can have lots of segments.

rte_path = os.path.join("..","..","transportation_data","dane-county","dane_wislr_rdwy_rte_link.csv")
with open(rte_path,'r') as f:
    rte_data = list(csv.reader(f))
rte_header = rte_data[0]
rte_data = rte_data[1:] 

# rte dict  {roadID : rteid}
rte_dict = {}
for r in rte_data:
    rte_dict[int(r[rte_header.index('RDWY_LINK_ID')])] =\
            int(r[rte_header.index('RDWY_RTE_ID')])

In [37]:
# create data
arc_list = []
for k in speed:
    # ref from ref to, rdwy_link_id, rte id
    arc_list.append([roadToRef[k][0],roadToRef[k][1],k,rte_dict[k]])
    
arc_dat = pd.DataFrame(arc_list,columns=["FROM","TO","RDWY_LINK_ID","RDWY_RTE_ID"])


In [76]:
arc_data = arc_dat.copy()
arc_data['use'] = 1

# some roads have same ref and end point. For convenience, we disregard it
arc_data.loc[arc_data["FROM"]==arc_data["TO"] ,'use'] = 0 

# create a column to store road turn
arc_data['turn_rteids'] = 0

# change FROM and TO to string
arc_data['FROM'] = arc_data['FROM'].astype(str)
arc_data['TO'] = arc_data['TO'].astype(str)

# start processing nodes

# 1. find all the unique nodes (unique reference points)
nodes = set(list(arc_data["FROM"]) + list(arc_data["TO"]))

intersection_index = 1
idx = 0
for N in nodes:
    
    if idx%100 == 0:
        print(idx)
        
    new_arcs = []
    
    # get the in nodes and out nodes
    out_N = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['TO'])
    #out_N_time = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['TIME'])
    out_N_id = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['RDWY_LINK_ID'])
    out_N_rteid = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['RDWY_RTE_ID'])
    in_N = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['FROM'])
    #in_N_time = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['TIME'])
    in_N_id = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['RDWY_LINK_ID'])
    in_N_rteid = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['RDWY_RTE_ID'])
    
    new_nodes = list(set(out_N + in_N))
    
    # store a node's corresponding node
    new_nodes_dict = {}
    for i in range(len(new_nodes)):
        new_nodes_dict[new_nodes[i]] = str(N) + "_"+ str(i + 1)
        
    # connnect new nodes with original nodes
    for index in range(len(out_N)):
        new_arcs.append([new_nodes_dict[out_N[index]],out_N[index],out_N_id[index],\
                         out_N_rteid[index],1,0])
        
    for index in range(len(in_N)):
        new_arcs.append([in_N[index],new_nodes_dict[in_N[index]],in_N_id[index],\
                         in_N_rteid[index],1,0])
    
    # connect new nodes with each other           
    for i in in_N:
        for o in out_N:
            # do not allow turn around
            if i != o:
                #turn
                if in_N_rteid[in_N.index(i)] != out_N_rteid[out_N.index(o)]:
                    new_arcs.append([new_nodes_dict[i],
                                     new_nodes_dict[o],
                                     'intersection_'+str(intersection_index),
                                     'intersection_'+str(intersection_index),
                                     1,
                                     (in_N_rteid[in_N.index(i)],out_N_rteid[out_N.index(o)])
                                     ])
                #go straight
                else:
                    new_arcs.append([new_nodes_dict[i],
                                     new_nodes_dict[o],
                                     'intersection_'+str(intersection_index),
                                     'intersection_'+str(intersection_index),
                                     1,
                                     0
                                    ])
                intersection_index += 1
                
    # disconnect original arcs
    arc_data.loc[(arc_data['FROM'] == N) | (arc_data['TO'] == N) , 'use'] = 0
    
    # add arcs to arc set
    new_arcs = pd.DataFrame(new_arcs, columns=["FROM","TO","RDWY_LINK_ID","RDWY_RTE_ID","use",'turn_rteids'])
    arc_data = arc_data.append(new_arcs,ignore_index=True)
    idx += 1
    
arc_data = arc_data[arc_data['use'] == 1]
arc_data = arc_data.reset_index(drop=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [78]:
arc_data.to_csv("turn_network.csv")

In [65]:
print(N)
print(out_N)
print(out_N_id)
print(out_N_rteid)

print('-------------')

print(in_N)
print(in_N_id)
print(in_N_rteid)


1673306
['1673116', '1673356', '1673354']
[4856414, 4835031, 4835032]
[395582, 395582, 395605]
-------------
['1673354', '1673116', '1673356']
[4856416, 4835030, 4856415]
[395605, 395582, 395582]


In [45]:
roadToRef[4850728]

('1662248', '1662248')

In [31]:
rte_dict

{4865668: 393605,
 4834480: 393605,
 4834474: 393605,
 4855860: 393605,
 4855859: 393605,
 5302117: 393605,
 5302118: 393605,
 4834475: 393605,
 4834476: 393605,
 4855858: 393605,
 4855864: 393605,
 4865669: 393605,
 4834530: 393606,
 4834531: 393606,
 4834532: 393606,
 4855916: 393606,
 4855915: 393606,
 4855914: 393606,
 4834063: 393607,
 4873079: 393607,
 4855476: 393607,
 4834152: 393607,
 4855536: 393607,
 4834092: 393607,
 4873080: 393607,
 4855447: 393607,
 4834265: 393608,
 4834264: 393608,
 4855648: 393608,
 4855649: 393608,
 4834183: 393609,
 4855567: 393609,
 5018118: 393610,
 4865683: 393610,
 4623596: 393610,
 4623597: 393610,
 4618453: 393610,
 4618452: 393610,
 4865684: 393610,
 5018117: 393610,
 4834052: 393611,
 4855436: 393611,
 5064230: 393612,
 5064229: 393612,
 4834506: 393613,
 4836439: 393613,
 4836440: 393613,
 4834093: 393613,
 5091170: 393613,
 5091171: 393613,
 4834095: 393613,
 4855479: 393613,
 5091172: 393613,
 5091173: 393613,
 4855477: 393613,
 4857823: 

In [18]:
roadToRef

{2156: ('10132', '10101'),
 2157: ('10101', '10091'),
 2159: ('10087', '10097'),
 2160: ('10097', '10016'),
 2161: ('10016', '10078'),
 2165: ('10125', '10079'),
 2166: ('10079', '10077'),
 2195: ('10204', '10200'),
 2196: ('10200', '10195'),
 2200: ('10240', '10238'),
 8436: ('12045', '12062'),
 8437: ('12062', '12063'),
 8443: ('11915', '11901'),
 8444: ('11901', '11895'),
 8445: ('11895', '11875'),
 8735: ('11763', '11717'),
 8736: ('11717', '11678'),
 8739: ('11506', '11439'),
 8740: ('11439', '11404'),
 8741: ('11404', '11343'),
 8762: ('9808', '9766'),
 9178: ('11882', '11871'),
 9194: ('11142', '11127'),
 9197: ('10817', '10787'),
 9200: ('10577', '10526'),
 9201: ('10526', '10470'),
 9958: ('11602', '11638'),
 9959: ('11638', '11652'),
 9962: ('11704', '11719'),
 9964: ('11739', '11751'),
 9965: ('11751', '11780'),
 9968: ('11898', '11962'),
 9970: ('11972', '12006'),
 11174: ('11988', '11987'),
 11176: ('11961', '11908'),
 11177: ('11908', '11885'),
 11182: ('11847', '11846'),

In [19]:
fnct_header

['RDWY_LINK_ID',
 'REF_SITE_FROM_ID',
 'REF_SITE_TO_ID',
 'LCM_FROM_TO_DIS',
 'LOC_FLKOS_FT',
 'LOC_TLKOS_FT',
 'ST_PRMY_SYMB_TY',
 'FNCT_CLS_CTGY_TYCD',
 'FNCT_CLS_CTGY_TYDC']